# Script to count Floops in a tensorflow model

In [43]:
import os
import tensorflow as tf
import numpy as np
def load_graph(frozen_graph_filename):

    # We load the protobuf file from the disk and parse it to retrieve the unserialized graph_def
    with tf.gfile.GFile(frozen_graph_filename, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    # Then, we import the graph_def into a new Graph and returns it
    with tf.Graph().as_default() as graph:
    # The name var will prefix every op/nodes in your graph
    # Since we load everything in a new graph, this is not needed
        tf.import_graph_def(graph_def, name="")
    return graph, graph_def

# DNN

In [14]:
# keras imports
from keras.models import Model, Sequential
from keras.layers import Dense, Input, Conv2D, Dropout, Flatten
from keras.layers import Concatenate, BatchNormalization, Activation
from keras.layers import MaxPooling2D, MaxPooling3D, GRU
from keras.utils import plot_model
from keras import regularizers
from keras import backend as K
from keras import metrics
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, TerminateOnNaN
from keras.regularizers import l1

In [2]:
#optmizer = 'adam'
# best model from optimization
DNN_neurons = 80
DNN_layers = 3
DNN_activation = 'elu'
dropout = 0.10
batch_size = 50
n_epochs = 500
labels = ['j_g', 'j_q', 'j_w', 'j_z', 'j_t']

In [4]:
#  model
def myModel():
    inputArray = Input(shape=(input_shape,))
    x = Dense(DNN_neurons, activation=DNN_activation, 
              kernel_initializer='lecun_uniform', name='dense_0')(inputArray)
    x = Dropout(dropout)(x)
    ####
    for i in range(1,DNN_layers):
        x = Dense(DNN_neurons, activation=DNN_activation, 
                  kernel_initializer='lecun_uniform', name='dense_%i' %i)(x)
        x = Dropout(dropout)(x)

    output = Dense(5, activation='softmax', kernel_initializer='lecun_uniform', 
                   name = 'output_softmax')(x)
    ####
    model = Model(inputs=inputArray, outputs=output)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
    return model

In [6]:
input_shape = len([12, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 48, 52])
thisModel = myModel()
thisModel.summary()

W0704 10:06:53.220622 4481996224 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0704 10:06:53.247987 4481996224 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0704 10:06:53.257430 4481996224 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0704 10:06:53.278475 4481996224 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0704 10:06:53.287894 4481996224 deprecati

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 16)                0         
_________________________________________________________________
dense_0 (Dense)              (None, 80)                1360      
_________________________________________________________________
dropout_1 (Dropout)          (None, 80)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 80)                6480      
_________________________________________________________________
dropout_2 (Dropout)          (None, 80)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 80)                6480      
_________________________________________________________________
dropout_3 (Dropout)          (None, 80)                0         
__________

In [44]:
# save model as protobuf


In [ ]:
# number of FLops
graph, graph_def = load_graph('DDB.pb')
db = graph.get_tensor_by_name('input_1:0')
cpf = graph.get_tensor_by_name('input_2:0')
sv = graph.get_tensor_by_name('input_3:0')
db_zeros = np.zeros([1, 1, 27])
cpf_zeros = np.zeros([1, 60, 8])
sv_zeros = np.zeros([1, 5, 2])
batchnorm = graph.get_tensor_by_name('db_input_batchnorm/keras_learning_phase:0')
pred = graph.get_tensor_by_name('ID_pred/Softmax:0')
sess = tf.Session(graph=graph)
run_metadata = tf.RunMetadata()
op = sess.graph.get_operations()
print([m.values() for m in op])
with graph.as_default():
#sess.run(tf.global_variables_initializer())
    result = sess.run(pred, feed_dict={db: db_zeros, cpf: cpf_zeros, sv: sv_zeros, batchnorm: False}, 
options=tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE),
run_metadata=run_metadata)
    flops = tf.profiler.profile(graph, options = tf.profiler.ProfileOptionBuilder.float_operation(),run_meta=run_metadata)
print('FLOP after freezing', flops.total_float_ops)

# CNN

In [7]:
CNN_filters = 10
CNN_filter_size = 3
CNN_MaxPool_size = 5
CNN_layers = 1
CNN_activation = 'elu'
DNN_neurons = 50
DNN_layers = 3
DNN_activation = 'elu'
dropout = 0.1
batch_size = 500
n_epochs = 500
labels = ['j_g', 'j_q', 'j_w', 'j_z', 'j_t']
nParticles = 50

In [8]:
#  model
def myModel():
    inputImage = Input(shape=(image_shape))
    x = Conv2D(CNN_filters, kernel_size=(CNN_filter_size,CNN_filter_size), 
               data_format="channels_last", strides=(1, 1), padding="same", input_shape=image_shape,
               kernel_initializer='lecun_uniform', name='cnn2D_0')(inputImage)
    x = BatchNormalization()(x)
    x = Activation(CNN_activation)(x)
    x = MaxPooling2D( pool_size = (CNN_MaxPool_size,CNN_MaxPool_size))(x)
    x = Dropout(dropout)(x)
    for i in range(1,CNN_layers):
        x = Conv2D(CNN_filters, kernel_size=(CNN_filter_size,CNN_filter_size), 
                   data_format="channels_last", strides=(1, 1), padding="same", input_shape=image_shape,
                   kernel_initializer='lecun_uniform', name='cnn2D_%i' %i)(x)
        x = BatchNormalization()(x)
        x = Activation(CNN_activation)(x)
        #x = MaxPooling2D( pool_size = (CNN_MaxPool_size,CNN_MaxPool_size))(x)
        x = Dropout(dropout)(x)
        
    ####
    x = Flatten()(x)
    #
    for i in range(DNN_layers):
        x = Dense(DNN_neurons, activation=DNN_activation, 
                  kernel_initializer='lecun_uniform', name='dense_%i' %i)(x)
        x = Dropout(dropout)(x)
    #
    output = Dense(5, activation='softmax', kernel_initializer='lecun_uniform', 
                   name = 'output_softmax')(x)
    ####
    model = Model(inputs=inputImage, outputs=output)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

    return model

In [9]:
img_rows = 100#X.shape[1]
img_cols = 100#X.shape[2]
image_shape = (img_rows, img_cols, 1)

In [10]:
thisModel = myModel()
thisModel.summary()

W0704 10:13:12.032739 4481996224 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:1834: The name tf.nn.fused_batch_norm is deprecated. Please use tf.compat.v1.nn.fused_batch_norm instead.

W0704 10:13:12.093631 4481996224 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 100, 100, 1)       0         
_________________________________________________________________
cnn2D_0 (Conv2D)             (None, 100, 100, 10)      100       
_________________________________________________________________
batch_normalization_1 (Batch (None, 100, 100, 10)      40        
_________________________________________________________________
activation_1 (Activation)    (None, 100, 100, 10)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 20, 20, 10)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 20, 20, 10)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4000)              0         
__________

# GRU

In [15]:
nParticles = 50
GRU_units= 50
DNN_neurons = 40
DNN_layers = 3
DNN_activation = 'relu'
dropout = 0.22
batch_size = 500
n_epochs = 50
#n_epochs = 1
labels = ['j_g', 'j_q', 'j_w', 'j_z', 'j_t']
input_shape = (nParticles,16)

In [16]:
#  model
def myModel():
    inputArray = Input(shape=(input_shape))
    x = GRU(GRU_units, activation='tanh',
            recurrent_activation='hard_sigmoid', name='gru')(inputArray)
    x = Dropout(dropout)(x)
    ####
    for i in range(0,DNN_layers):
        x = Dense(DNN_neurons, activation=DNN_activation, 
                  kernel_initializer='lecun_uniform', name='dense_%i' %i)(x)
        x = Dropout(dropout)(x)
    #
    output = Dense(5, activation='softmax', kernel_initializer='lecun_uniform', 
                   name = 'output_softmax')(x)
    ####
    model = Model(inputs=inputArray, outputs=output)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
    return model

In [17]:
thisModel = myModel()
thisModel.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 50, 16)            0         
_________________________________________________________________
gru (GRU)                    (None, 50)                10050     
_________________________________________________________________
dropout_8 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_0 (Dense)              (None, 40)                2040      
_________________________________________________________________
dropout_9 (Dropout)          (None, 40)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 40)                1640      
_________________________________________________________________
dropout_10 (Dropout)         (None, 40)                0         
__________

# JEDI-net

In [24]:
import torch
import torch.nn as nn
from torch.autograd.variable import *
import torch.optim as optim
import itertools

In [29]:
class GraphNet(nn.Module):
    def __init__(self, n_constituents, n_targets, params, hidden, De, Do, 
                 fr_activation=0, fo_activation=0, fc_activation=0, optimizer = 0, verbose = False):
        super(GraphNet, self).__init__()
        self.hidden = hidden
        self.P = len(params)
        self.N = n_constituents
        self.Nr = self.N * (self.N - 1)
        self.Dr = 0
        self.De = De
        self.Dx = 0
        self.Do = Do
        self.n_targets = n_targets
        self.fr_activation = fr_activation
        self.fo_activation = fo_activation
        self.fc_activation = fc_activation
        self.optimizer = optimizer
        self.verbose = verbose
        self.assign_matrices()

        self.Ra = torch.ones(self.Dr, self.Nr)
        self.fr1 = nn.Linear(2 * self.P + self.Dr, hidden)
        self.fr2 = nn.Linear(hidden, int(hidden/2))
        self.fr3 = nn.Linear(int(hidden/2), self.De)
        self.fo1 = nn.Linear(self.P + self.Dx + self.De, hidden)
        self.fo2 = nn.Linear(hidden, int(hidden/2))
        self.fo3 = nn.Linear(int(hidden/2), self.Do)
        self.fc1 = nn.Linear(self.Do * self.N, hidden)
        self.fc2 = nn.Linear(hidden, int(hidden/2))
        self.fc3 = nn.Linear(int(hidden/2), self.n_targets)

    def assign_matrices(self):
        self.Rr = torch.zeros(self.N, self.Nr)
        self.Rs = torch.zeros(self.N, self.Nr)
        receiver_sender_list = [i for i in itertools.product(range(self.N), range(self.N)) if i[0]!=i[1]]
        for i, (r, s) in enumerate(receiver_sender_list):
            self.Rr[r, i] = 1
            self.Rs[s, i] = 1
        self.Rr = Variable(self.Rr)
        self.Rs = Variable(self.Rs)

    def forward(self, x):
        Orr = self.tmul(x, self.Rr)
        Ors = self.tmul(x, self.Rs)
        B = torch.cat([Orr, Ors], 1)
        ### First MLP ###
        B = torch.transpose(B, 1, 2).contiguous()
        if self.fr_activation ==2:
            B = nn.functional.selu(self.fr1(B.view(-1, 2 * self.P + self.Dr)))
            B = nn.functional.selu(self.fr2(B))
            E = nn.functional.selu(self.fr3(B).view(-1, self.Nr, self.De))            
        elif self.fr_activation ==1:
            B = nn.functional.elu(self.fr1(B.view(-1, 2 * self.P + self.Dr)))
            B = nn.functional.elu(self.fr2(B))
            E = nn.functional.elu(self.fr3(B).view(-1, self.Nr, self.De))
        else:
            B = nn.functional.relu(self.fr1(B.view(-1, 2 * self.P + self.Dr)))
            B = nn.functional.relu(self.fr2(B))
            E = nn.functional.relu(self.fr3(B).view(-1, self.Nr, self.De))
        del B
        E = torch.transpose(E, 1, 2).contiguous()
        Ebar = self.tmul(E, torch.transpose(self.Rr, 0, 1).contiguous())
        del E
        C = torch.cat([x, Ebar], 1)
        del Ebar
        C = torch.transpose(C, 1, 2).contiguous()
        ### Second MLP ###
        if self.fo_activation ==2:
            C = nn.functional.selu(self.fo1(C.view(-1, self.P + self.Dx + self.De)))
            C = nn.functional.selu(self.fo2(C))
            O = nn.functional.selu(self.fo3(C).view(-1, self.N, self.Do))
        elif self.fo_activation ==1:
            C = nn.functional.elu(self.fo1(C.view(-1, self.P + self.Dx + self.De)))
            C = nn.functional.elu(self.fo2(C))
            O = nn.functional.elu(self.fo3(C).view(-1, self.N, self.Do))
        else:
            C = nn.functional.relu(self.fo1(C.view(-1, self.P + self.Dx + self.De)))
            C = nn.functional.relu(self.fo2(C))
            O = nn.functional.relu(self.fo3(C).view(-1, self.N, self.Do))
        del C
        ### Classification MLP ###
        if self.fc_activation ==2:
            N = nn.functional.selu(self.fc1(O.view(-1, self.Do * self.N)))
            N = nn.functional.selu(self.fc2(N))       
        elif self.fc_activation ==1:
            N = nn.functional.elu(self.fc1(O.view(-1, self.Do * self.N)))
            N = nn.functional.elu(self.fc2(N))
        else:
            N = nn.functional.relu(self.fc1(O.view(-1, self.Do * self.N)))
            N = nn.functional.relu(self.fc2(N))
        del O
        #N = nn.functional.relu(self.fc3(N))
        N = self.fc3(N)
        return N

    def tmul(self, x, y):  #Takes (I * J * K)(K * L) -> I * J * L 
        x_shape = x.size()
        y_shape = y.size()
        return torch.mm(x.view(-1, x_shape[2]), y).view(-1, x_shape[1], y_shape[1])

In [30]:
# ### Prepare Dataset
nParticles = 150
x = []
x.append(50) # hinned nodes
x.append(14) # De
x.append(6) # Do
x.append(2) # fr_activation_index
x.append(2) # fo_activation_index
x.append(0) # fc_activation_index
x.append(0) # optmizer_index

In [31]:
params = ['j1_px', 'j1_py' , 'j1_pz' , 'j1_e' , 'j1_erel' , 'j1_pt' , 'j1_ptrel', 'j1_eta' , 'j1_etarel' , 
          'j1_etarot' , 'j1_phi' , 'j1_phirel' , 'j1_phirot', 'j1_deltaR' , 'j1_costheta' , 'j1_costhetarel']

In [32]:
mymodel = GraphNet(nParticles, len(labels), params, int(x[0]), int(x[1]), int(x[2]), 
                       int(x[3]),  int(x[4]),  int(x[5]), int(x[6]), 0)

In [34]:
print(mymodel)
trainablePars = sum(p.numel() for p in mymodel.parameters() if p.requires_grad)
print('\nTrainable parameters:', trainablePars)

GraphNet(
  (fr1): Linear(in_features=32, out_features=50, bias=True)
  (fr2): Linear(in_features=50, out_features=25, bias=True)
  (fr3): Linear(in_features=25, out_features=14, bias=True)
  (fo1): Linear(in_features=30, out_features=50, bias=True)
  (fo2): Linear(in_features=50, out_features=25, bias=True)
  (fo3): Linear(in_features=25, out_features=6, bias=True)
  (fc1): Linear(in_features=900, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=25, bias=True)
  (fc3): Linear(in_features=25, out_features=5, bias=True)
)

Trainable parameters: 52725


# JEDI-net with Sum over O

In [38]:
class GraphNet(nn.Module):
    def __init__(self, n_constituents, n_targets, params, hidden, De, Do, 
                 fr_activation=0, fo_activation=0, fc_activation=0, optimizer = 0, verbose = False):
        super(GraphNet, self).__init__()
        self.hidden = hidden
        self.P = len(params)
        self.N = n_constituents
        self.Nr = self.N * (self.N - 1)
        self.Dr = 0
        self.De = De
        self.Dx = 0
        self.Do = Do
        self.n_targets = n_targets
        self.fr_activation = fr_activation
        self.fo_activation = fo_activation
        self.fc_activation = fc_activation
        self.optimizer = optimizer
        self.verbose = verbose
        self.assign_matrices()

        self.Ra = torch.ones(self.Dr, self.Nr)
        self.fr1 = nn.Linear(2 * self.P + self.Dr, hidden)
        self.fr2 = nn.Linear(hidden, int(hidden/2))
        self.fr3 = nn.Linear(int(hidden/2), self.De)
        self.fo1 = nn.Linear(self.P + self.Dx + self.De, hidden)
        self.fo2 = nn.Linear(hidden, int(hidden/2))
        self.fo3 = nn.Linear(int(hidden/2), self.Do)
        self.fc1 = nn.Linear(self.Do, hidden)
        self.fc2 = nn.Linear(hidden, int(hidden/2))
        self.fc3 = nn.Linear(int(hidden/2), self.n_targets)

    def assign_matrices(self):
        self.Rr = torch.zeros(self.N, self.Nr)
        self.Rs = torch.zeros(self.N, self.Nr)
        receiver_sender_list = [i for i in itertools.product(range(self.N), range(self.N)) if i[0]!=i[1]]
        for i, (r, s) in enumerate(receiver_sender_list):
            self.Rr[r, i] = 1
            self.Rs[s, i] = 1
        self.Rr = Variable(self.Rr)
        self.Rs = Variable(self.Rs)

    def forward(self, x):
        Orr = self.tmul(x, self.Rr)
        Ors = self.tmul(x, self.Rs)
        B = torch.cat([Orr, Ors], 1)
        ### First MLP ###
        B = torch.transpose(B, 1, 2).contiguous()
        if self.fr_activation ==2:
            B = nn.functional.selu(self.fr1(B.view(-1, 2 * self.P + self.Dr)))
            B = nn.functional.selu(self.fr2(B))
            E = nn.functional.selu(self.fr3(B).view(-1, self.Nr, self.De))            
        elif self.fr_activation ==1:
            B = nn.functional.elu(self.fr1(B.view(-1, 2 * self.P + self.Dr)))
            B = nn.functional.elu(self.fr2(B))
            E = nn.functional.elu(self.fr3(B).view(-1, self.Nr, self.De))
        else:
            B = nn.functional.relu(self.fr1(B.view(-1, 2 * self.P + self.Dr)))
            B = nn.functional.relu(self.fr2(B))
            E = nn.functional.relu(self.fr3(B).view(-1, self.Nr, self.De))
        del B
        E = torch.transpose(E, 1, 2).contiguous()
        Ebar = self.tmul(E, torch.transpose(self.Rr, 0, 1).contiguous())
        del E
        C = torch.cat([x, Ebar], 1)
        del Ebar
        C = torch.transpose(C, 1, 2).contiguous()
        ### Second MLP ###
        if self.fo_activation ==2:
            C = nn.functional.selu(self.fo1(C.view(-1, self.P + self.Dx + self.De)))
            C = nn.functional.selu(self.fo2(C))
            O = nn.functional.selu(self.fo3(C).view(-1, self.N, self.Do))
        elif self.fo_activation ==1:
            C = nn.functional.elu(self.fo1(C.view(-1, self.P + self.Dx + self.De)))
            C = nn.functional.elu(self.fo2(C))
            O = nn.functional.elu(self.fo3(C).view(-1, self.N, self.Do))
        else:
            C = nn.functional.relu(self.fo1(C.view(-1, self.P + self.Dx + self.De)))
            C = nn.functional.relu(self.fo2(C))
            O = nn.functional.relu(self.fo3(C).view(-1, self.N, self.Do))
        del C
        ## sum over the O matrix  
        O = torch.sum( O, dim=1)
        ### Classification MLP ###
        if self.fc_activation ==2:
            N = nn.functional.selu(self.fc1(O.view(-1, self.Do)))
            N = nn.functional.selu(self.fc2(N))       
        elif self.fc_activation ==1:
            N = nn.functional.elu(self.fc1(O.view(-1, self.Do)))
            N = nn.functional.elu(self.fc2(N))
        else:
            N = nn.functional.relu(self.fc1(O.view(-1, self.Do)))
            N = nn.functional.relu(self.fc2(N))
        del O
        #N = nn.functional.relu(self.fc3(N))
        N = self.fc3(N)
        return N

    def tmul(self, x, y):  #Takes (I * J * K)(K * L) -> I * J * L 
        x_shape = x.size()
        y_shape = y.size()
        return torch.mm(x.view(-1, x_shape[2]), y).view(-1, x_shape[1], y_shape[1])

In [39]:
# ### Prepare Dataset
nParticles = 50
x = []
x.append(50) # hinned nodes
x.append(12) # De
x.append(14) # Do
x.append(0) # fr_activation_index
x.append(0) # fo_activation_index
x.append(2) # fc_activation_index
x.append(0) # optmizer_index

In [40]:
mymodel = GraphNet(nParticles, len(labels), params, int(x[0]), int(x[1]), int(x[2]), 
                       int(x[3]),  int(x[4]),  int(x[5]), int(x[6]), 0)

In [41]:
print(mymodel)
trainablePars = sum(p.numel() for p in mymodel.parameters() if p.requires_grad)
print('\nTrainable parameters:', trainablePars)

GraphNet(
  (fr1): Linear(in_features=32, out_features=50, bias=True)
  (fr2): Linear(in_features=50, out_features=25, bias=True)
  (fr3): Linear(in_features=25, out_features=12, bias=True)
  (fo1): Linear(in_features=28, out_features=50, bias=True)
  (fo2): Linear(in_features=50, out_features=25, bias=True)
  (fo3): Linear(in_features=25, out_features=14, bias=True)
  (fc1): Linear(in_features=14, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=25, bias=True)
  (fc3): Linear(in_features=25, out_features=5, bias=True)
)

Trainable parameters: 8481
